In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [20]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [21]:
new_df = df_train
new_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [22]:
new_df.drop(columns = ['Ticket', 'Cabin', 'Name'], axis = 1, inplace = True)

In [23]:
new_df['Age'] = new_df.Age.fillna(new_df.Age.mean())
new_df['Fare'] = new_df.Fare.fillna(new_df.Fare.mean())

In [24]:
new_df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       2
dtype: int64

In [25]:
new_df.dropna(inplace = True)

In [26]:
from sklearn.preprocessing import OneHotEncoder 
ohe = OneHotEncoder()

In [27]:
sex_encoded = ohe.fit_transform(new_df[['Sex']]).toarray()
new_df[['female','male']] = sex_encoded

In [28]:
embarked_encoded = ohe.fit_transform(new_df[['Embarked']]).toarray()
new_df[['C','Q','S']] = embarked_encoded

In [30]:
new_df.drop(columns = ['Sex', 'Embarked'], axis = 1, inplace = True)

In [31]:
new_df.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,1,0,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,1.0
1,2,1,1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,0.0
2,3,1,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,1.0
3,4,1,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0
4,5,0,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0


In [32]:
from sklearn import tree
model = tree.DecisionTreeClassifier()

In [33]:
t_df = new_df[['Pclass','Age','SibSp','Parch','Fare','female','male','C','Q','S']]

In [34]:
model.fit(t_df, new_df.Survived)

DecisionTreeClassifier()

In [35]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [36]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [37]:
df_test.drop(columns = ['Ticket', 'Cabin', 'Name'], axis = 1, inplace = True)

In [38]:
df_test['Age'] = df_test.Age.fillna(df_test.Age.mean())
df_test['Fare'] = df_test.Fare.fillna(df_test.Fare.mean())

In [39]:
df_test.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [40]:
sex_encode = ohe.fit_transform(df_test[['Sex']]).toarray()
df_test[['female','male']] = sex_encode

In [41]:
embarked_encode = ohe.fit_transform(df_test[['Embarked']]).toarray()
df_test[['C','Q','S']] = embarked_encode

In [42]:
df_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,female,male,C,Q,S
0,892,3,male,34.5,0,0,7.8292,Q,0.0,1.0,0.0,1.0,0.0
1,893,3,female,47.0,1,0,7.0000,S,1.0,0.0,0.0,0.0,1.0
2,894,2,male,62.0,0,0,9.6875,Q,0.0,1.0,0.0,1.0,0.0
3,895,3,male,27.0,0,0,8.6625,S,0.0,1.0,0.0,0.0,1.0
4,896,3,female,22.0,1,1,12.2875,S,1.0,0.0,0.0,0.0,1.0


In [43]:
test_df = df_test[['Pclass','Age','SibSp','Parch','Fare','female','male','C','Q','S']]

In [47]:
submission_pred = model.predict(test_df)

In [48]:
test_id = df_test['PassengerId']

In [49]:
test_id

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [51]:
final_df = pd.DataFrame({ "PassengerId" : test_id.values,
                         "Survived" : submission_pred
}
)

In [52]:
final_df.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,1


In [53]:
final_df.to_csv('Final.csv', index=False)